# ライブラリーのインポート

In [ ]:
# # 形態素解析と共起ネットワークを描くためのインストール
# !pip install SudachiPy
# !pip install japanize-matplotlib
# !pip install pyvis
# !pip install spacy

# # 言語モデル(詳細については以下のurlを確認)
# pip install ja_core_news_sm
# pip install ja_core_news_md
# pip install ja_core_news_lg
# pip install ja_core_news_trf
# pip install ja-ginza
# pip install ja_ginza_electra
# pip install spacy_stanza

# 言語モデルの比較: https://zenn.dev/akimen/articles/8d818ca704f079#%E6%AF%94%E8%BC%83%E7%B5%90%E6%9E%9C

In [1]:
#前処理
import pandas as pd
import unicodedata
import re
import urllib.request

#共起ネットワーク
import numpy as np

import spacy
from spacy.pipeline import EntityRuler
from spacy import displacy

from sklearn.feature_extraction.text import CountVectorizer
from networkx.algorithms.community import greedy_modularity_communities

import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import japanize_matplotlib
from matplotlib.pyplot import figure, text

# 言語モデル(辞書)を定義
nlp = spacy.load('ja_ginza_electra') #GiNZA 日本語NLPライブラリ（transformerバージョン)らしい
# transformerとは: https://www.youtube.com/watch?v=QlfhLMUoJjw

# pandasの表示行数と列数を設定
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# データの読み込み

In [56]:
# テーブル読み込み
questionnaire_excel_file_path = r'xxx.xlsx'

df = pd.read_excel(questionnaire_excel_file_path, sheet_name='xxx')
df_whole = pd.read_excel(questionnaire_excel_file_path, sheet_name='yyy')

# 重複削除(※任意)

In [ ]:
# データ重複削除
display(len(df))
df = df.drop_duplicates('player_id')
display(len(df))

# 重複無し確認
df[df['player_id'].duplicated(keep=False)].sort_values('player_id')
df.to_excel(r"xxx.xlsx", index=False)

# 前処理

In [40]:
# 前処理の関数
def preprocess_texts(text):
    replaced_text = unicodedata.normalize("NFKC", text)
    replaced_text = replaced_text.upper()
    replaced_text = re.sub(r'[【】 () （） 『』　「」]', '', replaced_text) #【】 () 「」　『』の除去
    replaced_text = re.sub(r'[\[\]［］]', '', replaced_text)
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'\d+\.*\d*', '', replaced_text) # 数字を0にする
    return replaced_text

def nlp_preprocess(df):
    df.iloc[:, 2] = (
        df.iloc[:, 2]
        .str.replace('xxx', 'yyy')
        .apply(preprocess_texts)  # preprocess_texts 関数の呼び出し
    )

    return df

## ↓共起ネットワークに関する関数_開始

In [20]:
# 言語モデル
# nlp = spacy.load("ja_ginza_electra") #Transformerだからか時間がかかる
nlp = spacy.load("ja_ginza") #軽量に判定するためこちらを採用

# 各行のテキスト内で重複する単語を除去する関数
def remove_duplicate_words(text):
    doc = nlp(text)
    unique_words = set()
    filtered_tokens = []
    for token in doc:
        if token.text not in unique_words:
            unique_words.add(token.text)
            filtered_tokens.append(token.text)
    # 単語間にスペースを挿入して連結
    return ''.join(filtered_tokens)

# 上記の関数を適用
def process_text_column(df):
    df.iloc[:,2] = df.iloc[:,2].apply(remove_duplicate_words)
    return df

In [ ]:
# 辞書に単語を追加
# ファイルを読み込む
dictionary_path = r'xxx.txt'

with open(dictionary_path, 'r', encoding='utf-8') as file:
    content = file.read()

# 文字列をスペースで分割して単語リストにする
words = content.split()

# patterns リストを作成
patterns = [{"label": "NOUN", "pattern": word} for word in words]

config = {
   'overwrite_ents': True
}

# 辞書への追加(patterns リスト追加)
ruler = nlp.add_pipe('entity_ruler', config=config)
ruler.add_patterns(patterns)

In [ ]:
# 辞書がされているのか確認
doc = nlp('xxx')
doc = nlp(content)

# 単語依存構造の可視化
displacy.render(doc, style="dep", options={"compact":True},  jupyter=True)

# エンティティ抽出
displacy.render(doc, style="ent", options={"compact":True},  jupyter=True)

# 以下から辞書に登録されていることが確認できる

In [ ]:
# ストップワードをダウンロード&保存する関数
def download_stopwords(url, save_path):
    urllib.request.urlretrieve(url, save_path)

def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = [word.replace('\n', '') for word in file.readlines()]
    return stopwords

# ストップワードのURLと保存パス
stopword_url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
save_path = 'Japanese_Stopwords.txt'

# ストップワードのダウンロードと保存
download_stopwords(stopword_url, save_path)

# ストップワードの読み込み
japanese_stopwords = load_stopwords(save_path)

# データフレーム
df_japanese_stopwords = pd.DataFrame(japanese_stopwords, columns=['words'])

# ストップワードを追加する場合
stopwards_list = [['こと']]
japanese_stopwords.append(stopwards_list)

# 例: 読み込んだストップワードの表示
display(len(japanese_stopwords))
display(japanese_stopwords[:5])

In [ ]:
# 品詞を選択する関数
def extract_words(sent, pos_tags):
    words = [token.lemma_ for token in sent
             if token.pos_ in pos_tags and token.lemma_ ]
    return words

In [ ]:
# 共起行列を作成する関数
def count_cooccurrence(sents, token_length='{2,}'):
    token_pattern=f'\\b\\w{token_length}\\b'
    count_model = CountVectorizer(token_pattern=token_pattern)

    X = count_model.fit_transform(sents)
    words = count_model.get_feature_names_out()
    word_counts = np.asarray(X.sum(axis=0)).reshape(-1)

    X[X > 0] = 1
    Xc = (X.T * X)
    return words, word_counts, Xc, X

In [ ]:
# セグメント分けして文章作成する関数
def process_segment_texts(df, segment):
    df_segment_text = df[df['segment'] == segment]
    df_segment_text = '。'.join(df_segment_text.iloc[:, 2].astype(str))
    return df_segment_text

In [ ]:
# 単語とその頻度を算出する関数
def tokenize_and_calculate_cooccurrence(doc):
    # トークナイズの実行
    docs = []
    docs.append(nlp(doc))

    # 共起を算出
    # 品詞を指定します
    include_pos = ('NOUN', 'ADJ', 'PROPN')
    sents = []
    for doc in docs:
        sents.extend([' '.join(extract_words(sent, include_pos))
                    for sent in doc.sents])
    
    words, word_counts, Xc, X = count_cooccurrence(sents, token_length='{2,}')

    # NumPyの配列をDataFrameに変換
    df_words = pd.DataFrame(words, columns=['words'])
    df_word_counts = pd.DataFrame(word_counts, columns=['word_counts'])

    # df_slr_words と df_slr_word_counts を結合
    df_word_counts = pd.concat([df_words, df_word_counts], axis=1)
    df_word_counts = df_word_counts.sort_values('word_counts', ascending=False)

    df_slr_growth_rate = df_word_counts[~df_word_counts['words'].isin(df_japanese_stopwords['words'])].sort_values('word_counts', ascending=False)
    # df_slr_growth_rate = df_slr_growth_rate[df_slr_growth_rate['word_counts'] >= 2]

    return df_slr_growth_rate

In [ ]:
# 単語の共起情報を抽出する関数
def tokenize_and_extract_words(doc):
    # トークナイズの実行
    docs = []
    docs.append(nlp(doc))

    # 共起を算出
    # 品詞を指定します
    include_pos = ('NOUN', 'ADJ', 'PROPN')
    sents = []
    for doc in docs:
        sents.extend([' '.join(extract_words(sent, include_pos))
                    for sent in doc.sents])
    
    words, word_counts, Xc, X = count_cooccurrence(sents, token_length='{2,}')
    
    return words, word_counts, Xc, X

In [ ]:
# 各単語に対して重みを計算する関数
def word_weights(words, word_counts):
    count_max = word_counts.max()
    weights = [(word, {'weight': count / count_max})
               for word, count in zip(words, word_counts)]
    return weights

In [ ]:
# 共起行列から得られる単語の共起関係の重みを計算するための関数
def cooccurrence_weights(words, Xc, weight_cutoff):
    Xc_max = Xc.max()
    cutoff = weight_cutoff * Xc_max
    weights = [(words[i], words[j], Xc[i,j] / Xc_max)
               for i, j in zip(*Xc.nonzero()) if i < j and Xc[i,j] > cutoff]
    return weights

In [ ]:
# 共起ネットワークを作成する関数
def create_network(words, word_counts, Xc, weight_cutoff):
    G = nx.Graph()

    weights_w = word_weights(words, word_counts)
    G.add_nodes_from(weights_w)

    weights_c = cooccurrence_weights(words, Xc, weight_cutoff)
    G.add_weighted_edges_from(weights_c)

    G.remove_nodes_from(list(nx.isolates(G)))
    return G

In [ ]:
# NetworkX ライブラリを使用して与えられたグラフ G を可視化するための関数
def pyplot_network(G):
    plt.figure(figsize=(12, 8),dpi=300)
    if layout == 'spring':
      pos = nx.spring_layout(G, k=layout_parameter_k,iterations=50, threshold=0.0001, weight='weight', scale=1, center=None, dim=2, seed=123)

    elif layout == 'fruchterman_reingold':
      pos = nx.fruchterman_reingold_layout(G, dim=2, k=layout_parameter_k, pos=None, fixed=None, iterations=50, weight='weight', scale=1.0, center=None)

    elif layout == 'kamada_kawai':
      pos = nx.kamada_kawai_layout(G, dist=None, pos=None, weight=1, scale=1, center=None, dim=2)

    else:
      pos = nx.random_layout(G, center=None, dim=2, seed=None)

    connecteds = []
    colors_list = []

    for i, c in enumerate(greedy_modularity_communities(G)):
        connecteds.append(c)
        colors_list.append(1/20 * i)

    # ノードの色をカラーマップを使用して変更
    colors_array = cm.Pastel1(np.linspace(0.1, 0.9, len(colors_list)))

    node_colors = []
    for node in G.nodes():
        for i, c in enumerate(connecteds):
            if node in c:
                node_colors.append(colors_array[i])
                break

    weights_n = np.array(list(nx.get_node_attributes(G, 'weight').values()))
    nx.draw_networkx_nodes(G, pos, alpha=0.5,node_color=node_colors,node_size=7000 * weights_n) # alpha = 0.5

    weights_e = np.array(list(nx.get_edge_attributes(G, 'weight').values()))
    # nx.draw_networkx_edges(G, pos, edge_color="whitesmoke", width=20 * weights_e) # alpha = 0.5
    nx.draw_networkx_edges(G, pos, edge_color="lightblue", width=5 * weights_e)  # alpha = 0.5

    nx.draw_networkx_labels(G, pos, font_family='IPAexGothic')

    plt.axis("off")
    plt.show()

In [ ]:
df_words, df_word_counts, df_Xc, df_X = tokenize_and_extract_words(df_doc)

In [ ]:
# UIを使って、共起ネットワークの表示を変えられない理由
# └ Visual Studio Codeでは別で対話的にパラメータを選択する設定をしないといけないらしいから
#   └ Google Colab では、セル内のマジックコマンドを使って対話的にパラメータを選択できるが、VS Codeでは通常、スクリプト全体を実行するか、デバッグ機能を使用することが一般的

# @title 共起ネットワーク表示
layout = 'spring' #@param ['spring','fruchterman_reingold','kamada_kawai']
weight_cutoff = 0.09 #@param {type:"slider", min:0.005, max:0.20, step:0.005}
node_size_ = 500 #@param {type:"slider", min:1000, max:8000, step:500}
text_size = 13 #@param {type:"slider", min:5, max:30, step:1}
layout_parameter_k  = 0.3 #@param {type:"slider", min:0.01, max:1, step:0.01}

In [ ]:
# ネットワークの生成
G_df_slr = create_network(df_words, df_word_counts, df_Xc, weight_cutoff)

# 静的ネットワークの描画
pyplot_network(G_df_slr)

## ↑共起ネットワークに関する関数_終了

In [21]:
# セグメント別UUを求める関数
def uu_per_segment(df):
    
    # 満足度別回答数
    df_large = df.groupby(['xxx', 'segment'], dropna=False, as_index=False).count().sort_values(['segment'], ascending=False)
    
    # 回答数
    df_answer_cnt = pd.DataFrame([len(df)], columns=['回答数'])

    # 回答率
    df_answer_rate = pd.DataFrame([len(df) / len(df_whole)], columns=['回答率'])

    # セグメント別回答数
    df_segmnet_1_count = pd.DataFrame([df[df['segment'] == 'xxx'].iloc[:, 2].count()], columns=['xxx'])
    df_segmnet_2_count = pd.DataFrame([df[df['segment'] == 'yyy'].iloc[:, 2].count()], columns=['yyy'])
    df_segmnet_3_count = pd.DataFrame([df[df['segment'] == 'zzz'].iloc[:, 2].count()], columns=['zzz'])

    # 上記を結合
    df_small = pd.concat([df_answer_cnt, df_answer_rate, df_segmnet_1_count, df_segmnet_2_count, df_segmnet_3_count], axis=1)
    
    return df_large, df_small

In [22]:
# 満足度セグメントごとでデータフレームを作成する関数
def satisfaction_segment(df):

    # 満足度セグメント別回答数
    df_segmnet_1 = df[df['segment'] == 'xxx']
    df_segmnet_2 = df[df['segment'] == 'yyy']
    df_segmnet_3 = df[df['segment'] == 'zzz']
    
    return df, df_segmnet_1, df_segmnet_2, df_segmnet_3

In [ ]:
# データフレームの特定の列やセルに '\u0301' が含まれているかどうかを調べる手順
# 特定のセルや列に非ASCII文字が含まれているかを確認できる。

# 特定の列（例: 'columnName'）に含まれる '\u0301' を検索
offending_rows = df[df['columnName'].str.contains('\u0301', na=False)]

# 全体のデータフレームに対して検索
for column in df.columns:
    offending_rows = df[df[column].astype(str).str.contains('\u0301', na=False)]
    if not offending_rows.empty:
        print(f"Found '\u0301' in column '{column}'")

# 全体のデータフレームに対して検索（特定のセルを指定）
offending_rows = df[df.applymap(lambda x: '\u0301' in str(x)).any(axis=1)]

# UnicodeEncodeErrorが発生する行を特定し、その行をprint
for index, row in df_o.iterrows():
    try:
        row['xxx'].encode('shift_jis')
    except UnicodeEncodeError as e:
        print("UnicodeEncodeError occurred at index:", index)
        print("Row content:", row['xxx'])
        print("Error:", e)

### 例

In [38]:
df_copy = df.copy()

# 前処理
df_copy = nlp_preprocess(df_copy)

# 各行のテキスト内で重複する単語を除去する関数の実行
df_copy = process_text_column(df_copy)

# セグメント別UU
df_copy_large, df_copy_small = uu_per_segment(df_copy)

# セグメントごとでデータフレームを作成する関数
df_copy, df_segment_1, df_segment_2, df_segment_3 = satisfaction_segment(df_copy)

# データフレームの保存
df_copy_large.to_csv(r"xxx.csv", index=False, encoding='shift_jis')
df_copy_small.to_csv(r"yyy.csv", index=False, encoding='shift_jis')

df_copy.to_csv(r"xxx.csv", index=False, encoding='shift_jis')
df_segment_1.to_csv(r"xxx.csv", index=False, encoding='shift_jis')
df_segment_2.to_csv(r"xxx.csv", index=False, encoding='shift_jis')
df_segment_3.to_csv(r"xxx.csv", index=False, encoding='shift_jis')

## 参考文献_共起ネットワーク

1.Pythonで共起ネットワークを作る方法をご紹介します: https://note.com/noa813/n/ne506f884e467

2.(new)【Python】GoogleColab上でNetworkXによる日本語の共起ネットワークを文字化けせずにプロット: https://tkstock.site/2022/08/24/python-googlecolab-networkx-matplotlib-japanese-networkx-plot/

3.(old)Pythonを使って文章から共起ネットワークを作る　〜テキストマイニングでの可視化〜: https://www.dskomei.com/entry/2019/04/07/021028

4.言語モデル(+カスタムで辞書追加)について: https://zenn.dev/akimen/articles/8d818ca704f079

5.spaCyとGiNZAを使った日本語自然言語処理: https://qiita.com/wf-yamaday/items/3ffdcc15a5878b279d61

6.transformerとは: https://www.youtube.com/watch?v=QlfhLMUoJjw

7.NLP | GINZA v5で固有表現抽出のルール追加を試してみた: https://note.com/lizefield/n/n18fcac42afea

